# Importando bibliotecas

In [ ]:
!pip install mapclassify

In [55]:
import os
import osmnx as ox
import geopandas as gpd
import networkx as nx
import mapclassify

# Visualizando a base

In [42]:
gdf = gpd.read_file('pontos_onibus.gpkg', layer='SIRGAS_GPKG_pontoonibus')
if gdf.crs is None:
    # Se o CRS não estiver definido no arquivo, precisamos definir um.
    # SIRGAS 2000 (EPSG:31983) é uma suposição comum para dados geográficos do Brasil.
    print("CRS não definido no GeoPackage. Assumindo EPSG:31983 (SIRGAS 2000).")
    gdf = gdf.set_crs(epsg=31983, allow_override=True) # Use allow_override com cautela

# Converta para EPSG:4326 (lat/lon) para ox.graph_from_point
gdf = gdf.to_crs(epsg=4326)
print(f"CRS do gdf após leitura e conversão para EPSG:4326: {gdf.crs}")
gdf.tail()

CRS do gdf após leitura e conversão para EPSG:4326: EPSG:4326


,pt_id,pt_nome,pt_enderec,geometry
22169,50012403,"AV. PARAGUASSU PAULISTA , 711",HENRIQUE JACOBS/ R SANTA AFRA,POINT (-46.4918 -23.5373)
22170,740006475,"R. GASPAR BARRETO , 332",NELSON BALDINATO/ R ROSARIO DO CATETE,POINT (-46.5511 -23.59933)
22171,9412691,"R. MAL. ARGOLO FERRÃO , 550",None,POINT (-46.5712 -23.4981)
22172,560009202,"ESTR. ECOTURÍSTICA DE PARELHEIROS , 5518",AMERICO COXA/ R AMARO ALVES DO ROSARIO,POINT (-46.73455 -23.81731)
22173,740006506,"R. COSTA BARROS , 1984",MAJUBA/ R DOUTOR LAURINDO MINHOTO,POINT (-46.54928 -23.60105)


# Gerando o mapa em um raio de 5km do centro de SP

In [43]:
# 1) Defina paths dos caches
GRAPH_PATH = "cache_graph.graphml"
GDF_PATH   = "cache_gdf.gpkg"

# 2) Carregar ou criar/salvar grafo
if os.path.exists(GRAPH_PATH):
    print("🔄 Carregando grafo do cache…")
    G = ox.load_graphml(GRAPH_PATH)
else:
    print("⚙️ Construindo grafo do zero…")
    # (aqui copia todo o bloco de ox.graph_from_point + project_graph + peso time)
    centroid = gdf.unary_union.centroid
    G = ox.graph_from_point((centroid.y, centroid.x), dist=5000,
                            network_type='drive', simplify=False)
    G = ox.project_graph(G)
    avg_speed = 20000/3600
    for u, v, k, data in G.edges(keys=True, data=True):
        data['time'] = data['length']/avg_speed
    ox.save_graphml(G, GRAPH_PATH)
    print("✅ Grafo salvo em", GRAPH_PATH)

if os.path.exists(GDF_PATH):
    print("🔄 Carregando paradas cacheadas…")
    gdf = gpd.read_file(GDF_PATH)
else:
    print("⚙️ Mapeando paradas para nós OSM…")
    # carrega a base original
    gdf = gpd.read_file('pontos_onibus.gpkg', layer='SIRGAS_GPKG_pontoonibus')
    if gdf.crs is None:
        gdf = gdf.set_crs(epsg=31983)
    # projeta para lat/lon (pois vamos salvar o gdf em 4326 no cache pra folium)
    gdf = gdf.to_crs(epsg=4326)

    # *** projeta temporariamente pra CRS métrico do grafo antes de nearest_nodes ***
    # pega o CRS do grafo (salvo no atributo 'crs' ao project_graph)
    graph_crs = G.graph.get('crs')
    gdf_metric = gdf.to_crs(graph_crs)

    # mapeia cada ponto no nó mais próximo do grafo projetado
    gdf['node'] = gdf_metric.geometry.apply(
        lambda p: ox.nearest_nodes(G, X=p.x, Y=p.y))

    # salva em 4326 para recarregar depois
    gdf.to_file(GDF_PATH, driver="GPKG")
    print("✅ Paradas salvas em", GDF_PATH)

⚙️ Construindo grafo do zero…


C:\Users\Gabriel\AppData\Local\Temp\ipykernel_20980\419256780.py:12: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  centroid = gdf.unary_union.centroid


✅ Grafo salvo em cache_graph.graphml
⚙️ Mapeando paradas para nós OSM…
✅ Paradas salvas em cache_gdf.gpkg


# Carregando mapa em cache

In [47]:
# --- Carregar grafo e paradas do cache (rode sempre que reiniciar o kernel) ---
GRAPH_PATH = "cache_graph.graphml"
GDF_PATH   = "cache_gdf.gpkg"

# carrega em ~1s em vez de reconstruir tudo
G   = ox.load_graphml (GRAPH_PATH)
gdf = gpd.read_file  (GDF_PATH)

In [48]:
# --- TENTATIVA DE CORRIGIR TIPOS DO ATRIBUTO 'time' EM MEMÓRIA ---
print("Tentando corrigir os tipos do atributo 'time' no grafo G carregado...")
problemas_conversao = 0
arestas_sem_time = 0
arestas_corrigidas = 0

for u, v, k, data in G.edges(keys=True, data=True):
    if 'time' in data:
        if not isinstance(data['time'], (int, float)):
            try:
                data['time'] = float(data['time'])
                arestas_corrigidas +=1
            except (ValueError, TypeError):
                print(f"Não foi possível converter G.edges[{u},{v},{k}]['time'] (valor: {data['time']}, tipo: {type(data['time'])}) para float.")
                # Opção: atribuir um valor padrão alto para penalizar ou remover a aresta
                # data['time'] = float('inf') 
                problemas_conversao += 1
        # else: # O tipo já é numérico, não faz nada
            # print(f"Aresta ({u},{v},{k}) já tem 'time' numérico: {data['time']}")
    else:
        # Se 'time' não existe, e 'length' existe, calcula. Senão, penaliza.
        if 'length' in data and isinstance(data['length'], (int, float)):
            avg_speed = 20000/3600 # Certifique-se que avg_speed está definido ou defina aqui
            data['time'] = data['length'] / avg_speed
            print(f"Aresta ({u},{v},{k}) não tinha 'time', calculado a partir de 'length'.")
            arestas_corrigidas +=1
        else:
            # print(f"Aresta ({u},{v},{k}) não possui 'time' nem 'length' válida para cálculo.")
            # data['time'] = float('inf') # Penaliza arestas sem 'time' ou 'length'
            arestas_sem_time +=1


if problemas_conversao > 0:
    print(f"Alerta: {problemas_conversao} arestas tiveram problemas na conversão do atributo 'time'.")
if arestas_sem_time > 0:
    print(f"Alerta: {arestas_sem_time} arestas não tinham o atributo 'time' ou 'length' para cálculo (não foram corrigidas aqui, podem precisar de float('inf') se causar erro).")
if arestas_corrigidas > 0:
    print(f"Sucesso: {arestas_corrigidas} arestas tiveram seu atributo 'time' corrigido/adicionado em memória.")
if problemas_conversao == 0 and arestas_sem_time == 0 and arestas_corrigidas == 0:
    print("Nenhuma correção de tipo para 'time' pareceu necessária ou foi feita.")
else:
    print("Correção de tipos em memória concluída. Tente rodar a célula de teste novamente.")

# Opcional: verificar uma amostra após a tentativa de correção
# count = 0
# for u,v,k,data G.edges(keys=True, data=True):
# if 'time' in data and count < 5:
# print(f"Amostra pós-correção: Aresta ({u},{v},{k}) time: {data['time']}, tipo: {type(data['time'])}")
# count += 1
# elif count >=5:
# break

Tentando corrigir os tipos do atributo 'time' no grafo G carregado...
Sucesso: 82647 arestas tiveram seu atributo 'time' corrigido/adicionado em memória.
Correção de tipos em memória concluída. Tente rodar a célula de teste novamente.


## Listando todos os pontos de ônibus nesse raio

In [49]:
# --- listar quais pt_id estão dentro de 5 km do centro da área ---
# 1) Carrega original e projeta em métrico
orig_gdf =  gpd.read_file('pontos_onibus.gpkg', layer='SIRGAS_GPKG_pontoonibus')
if orig_gdf.crs is None:
    orig_gdf = orig_gdf.set_crs(epsg=31983)
orig_gdf = orig_gdf.to_crs(epsg=31983)

# 2) Calcula distância ao centróide
centroid_pt = orig_gdf.unary_union.centroid
orig_gdf['dist_m'] = orig_gdf.geometry.distance(centroid_pt)

# 3) Filtra em até 5000 m e mostra pt_id, nome e distância
inside = orig_gdf[orig_gdf['dist_m'] <= 5000]
display( inside[['pt_id','pt_nome','dist_m']].sort_values('dist_m') )

# 4) Se quiser só a lista de ids:
pt_ids = inside['pt_id'].tolist()
print("pt_id disponíveis:", pt_ids)

C:\Users\Gabriel\AppData\Local\Temp\ipykernel_20980\2124659832.py:9: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  centroid_pt = orig_gdf.unary_union.centroid


,pt_id,pt_nome,dist_m
9678,3305641,"R. OUVIDOR PORTUGAL , 506",61.821451
5491,3305935,"R. OUVIDOR PORTUGAL , 541",66.280505
14952,140016042,"R. ENG. PRUDENTE , 239",161.208304
9655,140016041,"R. ENG. PRUDENTE , 238",164.340121
6892,140016045,"R. DR. JOSÉ MARIA AZEVEDO , 254",263.983802
...,...,...,...
12997,570014302,"R. SILVA TELES , 1405",4991.768389
18766,10008762,"AV. VER. ABEL FERREIRA , 1560",4993.830562
18055,8209715,"R. TIJUCO PRETO , 131",4994.312931
15049,790015094,"R. CAPUTIRA , 118",4994.928268


pt_id disponíveis: [80014567, 800016522, 270016206, 9505653, 490017014, 2705242, 9206557, 80014082, 270016278, 6714619, 80014524, 570014314, 800016582, 570014291, 10014103, 800016537, 920016059, 540014257, 100014588, 80014595, 260016680, 1010092, 540014084, 950006678, 330014273, 790016969, 670016340, 330016066, 330010935, 3305889, 9505615, 9206186, 330010879, 5405607, 19064, 80014371, 270016089, 70016933, 920015258, 270016205, 10014222, 6714637, 100015927, 920016339, 80014110, 18876, 490017011, 18856, 6714500, 330010933, 950006675, 2600262, 70016928, 670012967, 5410183, 330010882, 920015263, 1410079, 330016005, 140016389, 540014169, 800016574, 3305800, 70001785, 140016043, 100014548, 100014330, 90013937, 670016554, 8010214, 800012673, 540014150, 800014545, 140014467, 9205573, 80014516, 10014096, 490016950, 80014482, 800016494, 670005370, 490017006, 9205549, 3305938, 950014191, 19057, 670014118, 330006950, 800016547, 800016594, 8010181, 3305877, 540014246, 950006613, 950006623, 54001423

## Testando o programa:
- Quando executar esse programa, basta colocar o pt_id de origem e o de destino, que o caminho ótimo será calculado e mostrado no mapa.

In [56]:
# --- células anteriores já carregaram G e gdf do cache ---

# 1) Receber input do usuário
orig_id = int(input("Digite o pt_id de origem: "))
dest_id = int(input("Digite o pt_id de destino: "))

# 2) Buscar os nós correspondentes
try:
    orig_node = gdf.loc[gdf.pt_id == orig_id, 'node'].iloc[0]
    dest_node = gdf.loc[gdf.pt_id == dest_id, 'node'].iloc[0]
    print(f"Nó de origem OSM: {orig_node}")
    print(f"Nó de destino OSM: {dest_node}")
except IndexError:
    print(f"Erro: pt_id de origem ({orig_id}) ou destino ({dest_id}) não encontrado no GeoDataFrame 'gdf'. Verifique os IDs.")
    # Interrompe a execução da célula se os IDs não forem válidos
    raise SystemExit("Encerrando devido a pt_id inválido.")


# 3) Calcular rota pelo Dijkstra
route = None
route_gdf = gpd.GeoDataFrame({'geometry': [], 'length': [], 'time': []},
                                  geometry='geometry', crs=G.graph.get('crs', 4326)) # Inicializa vazio

if orig_node == dest_node:
    print(f"Alerta: Nó de origem ({orig_node}) e destino ({dest_node}) são idênticos. Não há rota para traçar.")
else:
    try:
        route = nx.shortest_path(G, orig_node, dest_node, weight='time')
        print(f"Rota (lista de nós OSM): {route}")
        # Converter a rota (lista de nós) para um GeoDataFrame de arestas
        if len(route) > 1: # Só tenta converter se a rota tiver pelo menos 2 nós
            route_gdf = ox.routing.route_to_gdf(G, route, weight='time')
        else:
            print("Alerta: A rota calculada tem menos de 2 nós. Não é possível gerar GeoDataFrame de arestas.")

    except nx.NetworkXNoPath:
        print(f"Não foi encontrado caminho no grafo entre o nó {orig_node} e o nó {dest_node}.")
    except ValueError as e:
        # Este ValueError é o que você estava pegando de route_to_gdf
        print(f"ValueError ao converter rota para GeoDataFrame: {e}")
        print("Isso pode acontecer se a rota tiver apenas um nó ou se os nós não estiverem conectados como esperado.")
    except Exception as e:
        print(f"Ocorreu um erro inesperado ao calcular ou converter a rota: {e}")


# 4) Exibir no folium
# ... existing code ...
map_center_lat = gdf.geometry.y.mean()
map_center_lon = gdf.geometry.x.mean()
m = folium.Map(location=[map_center_lat, map_center_lon], zoom_start=13)

# Adicionar marcadores para os pontos de origem e destino
# Supondo que 'gdf' contém as geometrias dos pontos de ônibus e está em EPSG:4326
orig_geom = gdf.loc[gdf.pt_id == orig_id, 'geometry'].iloc[0]
dest_geom = gdf.loc[gdf.pt_id == dest_id, 'geometry'].iloc[0]

folium.Marker(
    location=[orig_geom.y, orig_geom.x],
    popup=f"Origem: {orig_id}",
    icon=folium.Icon(color="green", icon="play"),
).add_to(m)

folium.Marker(
    location=[dest_geom.y, dest_geom.x],
    popup=f"Destino: {dest_id}",
    icon=folium.Icon(color="red", icon="stop"),
).add_to(m)

# Adicionar a rota ao mapa usando o método .explore() do GeoDataFrame da rota
# .explore() irá re-projetar route_gdf para EPSG:4326 se necessário para o Folium
if not route_gdf.empty:
    m = route_gdf.explore(
        m=m,          # Mapa base existente
        color="blue", # Cor da rota
        linewidth=3,  # Largura da linha da rota
        tooltip=['name', 'length', 'time'], # Colunas para mostrar no tooltip (ajuste conforme os atributos da sua rota)
        legend=False
    )
else:
    print(f"Não foi possível plotar a rota de {orig_id} para {dest_id} (rota vazia ou não encontrada).")

m

Nó de origem OSM: 2383328445
Nó de destino OSM: 8917291535
Rota (lista de nós OSM): [2383328445, 7986708646, 2350165153, 2383328449, 7884112174, 25778479, 7986950207, 7986950206, 8959059094, 597444, 8959059089, 1666465199, 8959059084, 8685923354, 1666465192, 25778486, 25778539, 7980803115, 4356736821, 5306412693, 4356736829, 133456526, 9418436514, 133457072, 133457073, 8765461117, 8765441216, 133457074, 8765441215, 8765441214, 8762347564, 133457076, 1658806907, 1658806910, 7936413953, 133457078, 7936413952, 1658806918, 9416354273, 8762347562, 9418381012, 185638311, 185636117, 9418381006, 8953045055, 133460314, 5448240766, 5448240761, 174038864, 2530563739, 434003224, 9418436446, 9418436443, 434003222, 9418436445, 8943981837, 8953045044, 5110007254, 9184080239, 477360350, 8953045037, 9184080240, 477360349, 434003219, 5109992374, 12369709057, 5109945180, 5109992363, 2839446958, 461988041, 8670107710, 1972415710, 12369709062, 5306774977, 8670107704, 8670107709, 174039193, 5306767582, 5306